In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://tinyurl.com/ats1403")
df

,position,description
0,1.0,according to the financial times amazon compan...
1,1.0,more than staff to take part in stoppages aft...
2,1.0,each service claims to simplify your digital o...
3,1.0,the apple watch ultra has hit a new alltime lo...
4,1.0,save on the apple watch ultra at amazon via a...
...,...,...
1032,1.0,steve simons nikon z mastery online bootcamp f...
1033,1.0,lensbaby announced a new double glass ii opti...
1034,1.0,olympus om review in an amazing machine today...
1035,1.0,an old dining room buffet became counter space...


In [8]:
train_data = df.sample(frac=0.75)
test_data = df.drop(train_data.index)

up_words = " ".join(train_data[train_data["position"]==1]['description']).split()
down_words = " ".join(train_data[train_data["position"]==-1]['description']).split()

total_words = set(up_words + down_words)


num_up = len(train_data[train_data["position"]==1])
num_down = len(train_data[train_data["position"]==-1])

word_prob = {}
for word in total_words:
    word_prob[word] = {
        'prob_up': (up_words.count(word) + 0.5) / (num_up+1),
        'prob_down': (down_words.count(word) + 0.5) / (num_down+1)
    }

#word_prob

In [15]:
def predict(sentence):
    PDE_list = []
    PUE_list = []
    for word in sentence.split():
        if word in total_words:
            PEU = word_prob[word]["prob_up"]
            PED = word_prob[word]["prob_down"]
            PUE = (PEU*0.5) / (PEU*0.5 + PED*0.5)
            PDE = 1 - PUE
            PUE_list.append(PUE)
            PDE_list.append(PDE)

    log_PUE = sum(np.log10(PUE_list))
    log_PDE = sum(np.log10(PDE_list))

    if log_PUE > log_PDE:
        return 1
    else:
        return -1

In [16]:
correct = 0
incorrect = 0

for index, row in test_data.iterrows():
    prediction = predict(row["description"])
    if prediction==row["position"]:
        correct += 1
    else:
        incorrect += 1

accuracy = correct / (correct+incorrect)
accuracy

0.7644787644787645